**Imports:**

In [ ]:
import io, os, sys, types, datetime, math, StringIO, time
import json

import pandas
import igraph
import graphistry
import splunklib.client as client
import splunklib.results as results

**API Connections:**

In [ ]:
try:
    graphistry.register('$api_key$')
    print "\n✨ Graphistry API Connection - OK"
except Exception,e: print str(e)
    
# Create a Service instance and Attempt Connection to Splunk
cargs = {
    'host': '$splunk_ip$',
    'scheme': 'https',
    'port': 8089,
    'username': '$username$',
    'password': '$password$'
}

try:
    service = client.connect(**cargs)
    print "✨ Splunk API Connection - OK"
except Exception,e: print str(e)

**Splunk Variables and Search Query:**

In [ ]:
#Splunk Variables
kwargs={"exec_mode": "normal"}

kwargs_options={"output_mode": "csv",
                "count": "100000",
                "fields": ["user.screen_name", "in_reply_to_screen_name"]}

#Splunk search query
splunk_query="""
search index="1337list" in_reply_to_screen_name!=null earliest=-7d
| table "user.screen_name", "in_reply_to_screen_name"
"""

#Graphistry variables
source = 'user.screen_name'
destination = 'in_reply_to_screen_name'
node = 'user.screen_name'

**Define a Function to Query Splunk and Display Search Progress:**

In [ ]:
# Function to Perform a Splunk search
def execute_query():
    
    # Execute Search
    job = service.jobs.create(splunk_query, **kwargs)

    # A normal search returns the job's SID right away, so we need to poll for completion
    while True:
        while not job.is_ready():
            pass
        stats = {"isDone": job["isDone"],
                 "doneProgress": float(job["doneProgress"])*100,
                 "scanCount": int(job["scanCount"]),
                 "eventCount": int(job["eventCount"]), 
                 "resultCount": int(job["resultCount"])}
        
        status = ("\r%(doneProgress)03.1f%%   %(scanCount)d scanned   " 
                  "%(eventCount)d matched   %(resultCount)d results") % stats

        sys.stdout.write(status)
        sys.stdout.flush()
        if stats["isDone"] == "1":
            print "\nSearch duration: ", job["runDuration"], "seconds"
            sys.stdout.write("\n***DONE***")
            break
        time.sleep(0.5)

    # Get the results and display them
    csv_results = job.results(**kwargs_options).read()
    job.cancel()
    return csv_results

**Execute Splunk Query:**

In [ ]:
csv_results = execute_query()

**Store Results in Pandas Dataframe:**

In [ ]:
csv_results_pandas = pandas.read_csv(StringIO.StringIO(csv_results), encoding='utf8', sep=',', low_memory=False)

In [ ]:
type(csv_results_pandas), csv_results_pandas.shape

**Display Sample of Results:**

In [ ]:
csv_results_pandas.head(5)

**Define Graphistry Visual Schema:**

In [ ]:
g = graphistry.bind(source=source, destination=destination, node=node)

**--->**  **Bind Data and Submit to Plot:**  **<---**

In [ ]:
g.plot(csv_results_pandas)

**Convert to iGraph for Additional Analysis:**

In [ ]:
ig = g.pandas2igraph(csv_results_pandas)

**Directed:**

In [ ]:
ig.vs['pagerank'] = ig.pagerank(directed=False)
ig.vs['betweenness'] = ig.betweenness(directed=True)
ig.es['ebetweenness'] = ig.edge_betweenness(directed=True)

**Undirected:**

In [ ]:
uig = ig.copy()
uig.to_undirected()
ig.vs['community_infomap'] = uig.community_infomap().membership
ig.vs['community_louvain'] = uig.community_multilevel().membership
igraph.summary(ig)

**--->**  **Bind Data and Re-submit to Plot:**  **<---**

In [ ]:
g.bind(point_color='', point_size='betweenness').plot(ig)